In [1]:
import os
import json
import warnings
import numpy as np
from torch.utils.data import Dataset

In [2]:
def pc_normalize(pc):
    centroid = np.mean(pc, axis=0)
    pc = pc - centroid
    m = np.max(np.sqrt(np.sum(pc ** 2, axis=1)))
    pc = pc / m
    return pc

In [7]:
class PartNormalDataset(Dataset):
    def __init__(self,root = '/data1/linfeng/shapenetcore_partanno_segmentation_benchmark_v0_normal', npoints=2500, split='train', class_choice=None, normal_channel=True):
        self.npoints = npoints
        self.root = root
        self.catfile = os.path.join(self.root, 'synsetoffset2category.txt')
        self.cat = {}
        self.normal_channel = normal_channel


        with open(self.catfile, 'r') as f:
            for line in f:
                ls = line.strip().split()
                self.cat[ls[0]] = ls[1]
        self.cat = {k: v for k, v in self.cat.items()}
        self.classes_original = dict(zip(self.cat, range(len(self.cat))))

        if not class_choice is  None:
            self.cat = {k:v for k,v in self.cat.items() if k in class_choice}
        # print(self.cat)

        self.meta = {}
        with open(os.path.join(self.root, 'train_test_split', 'shuffled_train_file_list.json'), 'r') as f:
            train_ids = set([str(d.split('/')[2]) for d in json.load(f)])
        with open(os.path.join(self.root, 'train_test_split', 'shuffled_val_file_list.json'), 'r') as f:
            val_ids = set([str(d.split('/')[2]) for d in json.load(f)])
        with open(os.path.join(self.root, 'train_test_split', 'shuffled_test_file_list.json'), 'r') as f:
            test_ids = set([str(d.split('/')[2]) for d in json.load(f)])
        for item in self.cat:
            # print('category', item)
            self.meta[item] = []
            dir_point = os.path.join(self.root, self.cat[item])
            fns = sorted(os.listdir(dir_point))
            # print(fns[0][0:-4])
            if split == 'trainval':
                fns = [fn for fn in fns if ((fn[0:-4] in train_ids) or (fn[0:-4] in val_ids))]
            elif split == 'train':
                fns = [fn for fn in fns if fn[0:-4] in train_ids]
            elif split == 'val':
                fns = [fn for fn in fns if fn[0:-4] in val_ids]
            elif split == 'test':
                fns = [fn for fn in fns if fn[0:-4] in test_ids]
            else:
                print('Unknown split: %s. Exiting..' % (split))
                exit(-1)

            # print(os.path.basename(fns))
            for fn in fns:
                token = (os.path.splitext(os.path.basename(fn))[0])
                self.meta[item].append(os.path.join(dir_point, token + '.txt'))

        self.datapath = []
        for item in self.cat:
            for fn in self.meta[item]:
                self.datapath.append((item, fn))

        self.classes = {}
        for i in self.cat.keys():
            self.classes[i] = self.classes_original[i]

        # Mapping from category ('Chair') to a list of int [10,11,12,13] as segmentation labels
        self.seg_classes = {'Earphone': [16, 17, 18], 'Motorbike': [30, 31, 32, 33, 34, 35], 'Rocket': [41, 42, 43],
                            'Car': [8, 9, 10, 11], 'Laptop': [28, 29], 'Cap': [6, 7], 'Skateboard': [44, 45, 46],
                            'Mug': [36, 37], 'Guitar': [19, 20, 21], 'Bag': [4, 5], 'Lamp': [24, 25, 26, 27],
                            'Table': [47, 48, 49], 'Airplane': [0, 1, 2, 3], 'Pistol': [38, 39, 40],
                            'Chair': [12, 13, 14, 15], 'Knife': [22, 23]}

        # for cat in sorted(self.seg_classes.keys()):
        #     print(cat, self.seg_classes[cat])

        self.cache = {}  # from index to (point_set, cls, seg) tuple
        self.cache_size = 20000


    def __getitem__(self, index):
        if index in self.cache:
            point_set, cls, seg = self.cache[index]
        else:
            fn = self.datapath[index]
            cat = self.datapath[index][0]
            cls = self.classes[cat]
            cls = np.array([cls]).astype(np.int32)
            data = np.loadtxt(fn[1]).astype(np.float32)
            if not self.normal_channel:
                point_set = data[:, 0:3]
            else:
                point_set = data[:, 0:6]
            seg = data[:, -1].astype(np.int32)
            if len(self.cache) < self.cache_size:
                self.cache[index] = (point_set, cls, seg)
        point_set[:, 0:3] = pc_normalize(point_set[:, 0:3])

        choice = np.random.choice(len(seg), self.npoints, replace=True)
        # resample
        point_set = point_set[choice, :]
        seg = seg[choice]

        return point_set, cls, seg

    def __len__(self):
        return len(self.datapath)

In [31]:
m = PartNormalDataset(npoints=8192,normal_channel=True)

In [32]:
_,cls,x=m[10000]

In [30]:
cls

array([15], dtype=int32)

In [2]:
import random

In [4]:
x = random.randint(0,10)

In [5]:
x

2

In [2]:
import json
data_dir = "/data2/llf/partnet/data_v0/"
data_list = os.listdir(data_dir)
data_path = os.path.join(data_dir,data_list[100])
result_path = os.path.join(data_path,"result.json")
result_merge_path = os.path.join(data_path,"result_after_merging.json")
with open(result_path, 'r') as fcc_file:
     result= json.load(fcc_file)
with open(result_merge_path, 'r') as fcc_file:
     result_merge= json.load(fcc_file)


In [24]:
result[0]['name']

'table'

In [34]:
for i in result[0]['children']:
    print(i)

{'text': 'Desk', 'children': [{'text': 'Tabletop', 'children': [{'text': 'Tabletop surface', 'children': [{'text': 'Board', 'objs': ['new-14'], 'id': 45, 'name': 'board'}], 'id': 44, 'name': 'tabletop_surface'}], 'id': 2, 'name': 'tabletop'}, {'text': 'Drawer Base', 'children': [{'text': 'Vertical side panel', 'objs': ['new-9'], 'id': 26, 'name': 'vertical_side_panel'}, {'text': 'Vertical side panel', 'objs': ['new-13', 'new-12'], 'id': 27, 'name': 'vertical_side_panel'}, {'text': 'Vertical side panel', 'objs': ['new-8'], 'id': 28, 'name': 'vertical_side_panel'}, {'text': 'Vertical side panel', 'objs': ['new-11', 'new-10'], 'id': 29, 'name': 'vertical_side_panel'}, {'text': 'Leg', 'objs': ['new-4', 'original-14'], 'id': 30, 'name': 'leg'}, {'text': 'Leg', 'objs': ['new-5'], 'id': 31, 'name': 'leg'}, {'text': 'Leg', 'objs': ['new-2'], 'id': 32, 'name': 'leg'}, {'text': 'Leg', 'objs': ['new-3', 'original-21'], 'id': 33, 'name': 'leg'}, {'text': 'Bar Stretcher', 'objs': ['new-1'], 'id': 3

In [13]:
import h5py
data_dir = "/data2/llf/partnet/sem_seg_h5"
class_dir = os.listdir(data_dir)
for class_name in class_dir:
    f = h5py.File(os.path.join(data_dir,class_name,"train-00.h5"), 'r')
    # for key in f.keys():
    #     print(f[])
    print(f['label_seg'][0])
    break

[29 13 29 ... 26 13 13]


In [27]:
set(f['label_seg'].reshape(-1))

AttributeError: 'Dataset' object has no attribute 'data'

In [1]:
import json
import os
data_dir = "/data1/linfeng/partnet-mobility"
count=0
valid=0
all_class=set()
valid_class=set()
for id in os.listdir(data_dir):
    with open(os.path.join(data_dir,id,"meta.json"), 'r') as fcc_file:
        cls = json.load(fcc_file)['model_cat']
    if os.path.exists(os.path.join(data_dir,id,"point_sample")):
        valid_class.add(cls)
        valid+=1
    all_class.add(cls)
    count+=1

In [8]:
count

2347

In [9]:
valid

1094

In [12]:
len(all_class)

46

In [13]:
len(valid_class)

19

In [14]:
!pip install sapien

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.1/39.1 MB 1.8 MB/s eta 0:00:00m eta 0:00:010:00:01


In [15]:
valid_class

{'Bottle',
 'Box',
 'Chair',
 'Clock',
 'Dishwasher',
 'Display',
 'Door',
 'Faucet',
 'Kettle',
 'Keyboard',
 'Lamp',
 'Laptop',
 'Microwave',
 'Oven',
 'Refrigerator',
 'Scissors',
 'StorageFurniture',
 'Table',
 'TrashCan'}